# SPFA算法详解及C++模板

## 算法概述

SPFA（Shortest Path Faster Algorithm，最短路径快速算法）是Bellman-Ford算法的队列优化版本，用于求解带权有向图中单源最短路径问题。该算法在大多数情况下具有较高的效率，特别是在处理稀疏图时表现优异。

## 算法特点

- **时间复杂度**：平均情况O(m)，最坏情况O(nm)（n为顶点数，m为边数）
- **适用范围**：带权有向图，可以处理负权边，但**不能处理负权环**
- **优势**：在稀疏图上效率高于Dijkstra算法，能检测负权环

## 算法步骤

1. 初始化距离数组dist，源点距离为0，其他点为无穷大
2. 创建队列，将源点入队
3. 从队列中取出顶点u，遍历u的所有邻接边
4. 对每条边(u, v, w)，如果dist[u] + w < dist[v]，则更新dist[v] = dist[u] + w
5. 如果v不在队列中，将v入队
6. 重复步骤3-5直到队列为空

## C++模板实现

```cpp
#include <iostream>
#include <vector>
#include <queue>
#include <climits>
using namespace std;

const int INF = INT_MAX;

struct Edge {
    int to;
    int weight;
    Edge(int t, int w) : to(t), weight(w) {}
};

class SPFA {
private:
    int n; // 顶点数
    vector<vector<Edge>> graph; // 邻接表
    vector<int> dist; // 最短距离
    vector<bool> inQueue; // 是否在队列中
    vector<int> count; // 入队次数，用于检测负环
    
public:
    SPFA(int numNodes) : n(numNodes) {
        graph.resize(n);
        dist.resize(n, INF);
        inQueue.resize(n, false);
        count.resize(n, 0);
    }
    
    // 添加边
    void addEdge(int from, int to, int weight) {
        graph[from].emplace_back(to, weight);
    }
    
    // 计算最短路径
    bool shortestPath(int start) {
        dist[start] = 0;
        queue<int> q;
        q.push(start);
        inQueue[start] = true;
        count[start]++;
        
        while (!q.empty()) {
            int u = q.front();
            q.pop();
            inQueue[u] = false;
            
            for (const Edge& edge : graph[u]) {
                int v = edge.to;
                int w = edge.weight;
                
                if (dist[u] != INF && dist[u] + w < dist[v]) {
                    dist[v] = dist[u] + w;
                    
                    if (!inQueue[v]) {
                        q.push(v);
                        inQueue[v] = true;
                        count[v]++;
                        
                        // 检测负环：如果顶点入队次数超过n次，说明存在负环
                        if (count[v] > n) {
                            return false; // 存在负环
                        }
                    }
                }
            }
        }
        
        return true; // 不存在负环
    }
    
    // 获取最短距离
    int getDistance(int node) {
        return dist[node];
    }
};

// 使用示例
int main() {
    int n = 5; // 顶点数
    SPFA spfa(n);
    
    // 添加边
    spfa.addEdge(0, 1, 5);
    spfa.addEdge(0, 2, 3);
    spfa.addEdge(1, 2, 2);
    spfa.addEdge(1, 3, 6);
    spfa.addEdge(2, 3, 7);
    spfa.addEdge(3, 4, -3);
    spfa.addEdge(2, 4, 4);
    
    int start = 0;
    if (spfa.shortestPath(start)) {
        cout << "从顶点 " << start << " 到各顶点的最短距离：" << endl;
        for (int i = 0; i < n; i++) {
            int d = spfa.getDistance(i);
            if (d == INF) {
                cout << "顶点 " << i << ": 不可达" << endl;
            } else {
                cout << "顶点 " << i << ": " << d << endl;
            }
        }
    } else {
        cout << "图中存在负权环，无法计算最短路径" << endl;
    }
    
    return 0;
}
```

## 算法优化

### 1. SLF优化（Small Label First）
```cpp
// 使用双端队列，将新节点与队首比较
deque<int> dq;
// 在push时优化
if (!dq.empty() && dist[v] < dist[dq.front()]) {
    dq.push_front(v);
} else {
    dq.push_back(v);
}
```

### 2. LLL优化（Large Label Last）
```cpp
// 维护队列中距离的平均值
long long sum = 0;
int cnt = 0;

// 入队时
sum += dist[v];
cnt++;

// 出队时
while (!dq.empty() && dist[dq.front()] > sum / cnt) {
    dq.push_back(dq.front());
    dq.pop_front();
}
```

## 注意事项

1. **负环检测**：SPFA可以检测负环，通过记录每个顶点的入队次数，如果超过n次则存在负环
2. **性能问题**：在最坏情况下时间复杂度为O(nm)，可能被特殊数据卡掉
3. **稀疏图优势**：在边数较少的图中表现优异
4. **稠密图考虑**：对于稠密图，Dijkstra算法可能更稳定

## 适用场景

- 图中存在负权边
- 需要检测负环
- 稀疏图的最短路径计算
- 对算法稳定性要求不高的情况

这个模板提供了SPFA算法的基本实现，可以根据具体需求进行相应的优化和调整。